# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [41]:
import nltk
from nltk.corpus import brown, reuters, gutenberg, webtext

# Download necessary NLTK data and external corpus
nltk.download(['brown', 'reuters', 'gutenberg', 'webtext', 'punkt', 'genesis'])

[nltk_data] Downloading package brown to /usr/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to /usr/share/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package gutenberg to /usr/share/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package webtext to /usr/share/nltk_data...
[nltk_data]   Package webtext is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package genesis to /usr/share/nltk_data...
[nltk_data]   Package genesis is already up-to-date!


True

In [42]:
import requests

# Download and load Norvig's big.txt
response = requests.get('https://norvig.com/big.txt')
with open('big.txt', 'w') as f:
    f.write(response.text.lower())

In [43]:
from nltk.tokenize import RegexpTokenizer

def load_corpus():
    """Load NLTK corpora and external data."""
    raw_text = []
    # NLTK corpora
    for corpus in [brown, reuters, gutenberg, webtext]:
        try:
            raw_text.extend([corpus.raw(file_id) for file_id in corpus.fileids()])
        except LookupError:
            continue
    # External data (big.txt)
    with open('big.txt', 'r') as f:
        raw_text.append(f.read())
    return ' '.join(raw_text).lower()

# Load corpus and tokenize with improved tokenizer
tokenizer = RegexpTokenizer(r"\w+[\w']*")  # Handles contractions
text = load_corpus()
words = tokenizer.tokenize(text)

In [44]:
from nltk.util import ngrams
from collections import Counter

# Build n-grams with Laplace smoothing
unigrams = Counter(words)
bigrams = Counter(ngrams(words, 2))
trigrams = Counter(ngrams(words, 3))

In [45]:
import re
from collections import defaultdict

keyboard_neighbors = {
    'q': ['w', 'a', 's'],
    'w': ['q', 'e', 'a', 's', 'd'],
    'e': ['w', 'r', 's', 'd', 'f'],
    'r': ['e', 't', 'd', 'f', 'g'],
    't': ['r', 'y', 'f', 'g', 'h'],
    'y': ['t', 'u', 'g', 'h', 'j'],
    'u': ['y', 'i', 'h', 'j', 'k'],
    'i': ['u', 'o', 'j', 'k', 'l'],
    'o': ['i', 'p', 'k', 'l'],
    'p': ['o', 'l'],
    'a': ['q', 'w', 's', 'z', 'x'],
    's': ['a', 'w', 'e', 'd', 'z', 'x', 'c'],
    'd': ['s', 'e', 'r', 'f', 'x', 'c', 'v'],
    'f': ['d', 'r', 't', 'g', 'c', 'v', 'b'],
    'g': ['f', 't', 'y', 'h', 'v', 'b', 'n'],
    'h': ['g', 'y', 'u', 'j', 'b', 'n', 'm'],
    'j': ['h', 'u', 'i', 'k', 'n', 'm'],
    'k': ['j', 'i', 'o', 'l', 'm'],
    'l': ['k', 'o', 'p'],
    'z': ['a', 's', 'x'],
    'x': ['z', 's', 'd', 'c'],
    'c': ['x', 'd', 'f', 'v'],
    'v': ['c', 'f', 'g', 'b'],
    'b': ['v', 'g', 'h', 'n'],
    'n': ['b', 'h', 'j', 'm'],
    'm': ['n', 'j', 'k']
}

def edits1(word):
    """Edits with priority to keyboard neighbors."""
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    edits = set()
    # Deletes, transposes, replaces, inserts
    for L, R in splits:
        if R: edits.add(L + R[1:])  # Deletes
        if len(R) > 1: edits.add(L + R[1] + R[0] + R[2:])  # Transposes
        if R:
            for c in keyboard_neighbors.get(R[0], 'abcdefghijklmnopqrstuvwxyz'):
                edits.add(L + c + R[1:])  # Prioritize nearby keys
        for c in 'abcdefghijklmnopqrstuvwxyz':
            edits.add(L + c + R)  # Inserts
    return edits

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

def known(words):
    return {w for w in words if w in unigrams}

def candidates(word):
    return known([word]) or known(edits1(word)) or known(edits2(word)) or [word]

def correct_word(word, prev=None, next=None):
    """Use interpolated probability of n-grams."""
    candidates_list = candidates(word)
    if not candidates_list:
        return word
    # Linear interpolation: trigram*0.6 + bigram*0.3 + unigram*0.1
    scores = {}
    for cand in candidates_list:
        trigram_score = trigrams.get((prev, cand, next), 0) * 0.6 if prev and next else 0
        bigram_prev = bigrams.get((prev, cand), 0) * 0.3 if prev else 0
        bigram_next = bigrams.get((cand, next), 0) * 0.3 if next else 0
        unigram_score = unigrams[cand] * 0.1
        scores[cand] = trigram_score + bigram_prev + bigram_next + unigram_score
    return max(scores, key=scores.get)

def correct_text(text):
    tokens = tokenizer.tokenize(text.lower())
    corrected = []
    for i, word in enumerate(tokens):
        prev = tokens[i-1] if i > 0 else None
        next_word = tokens[i+1] if i < len(tokens)-1 else None
        corrected.append(correct_word(word, prev, next_word))
    return ' '.join(corrected)

neve gonna give you up


In [46]:
# Test
print(correct_text("doing sport"))

doing sport


In [50]:
text_with_errors = "gonna plau duing sportd"
corrected_text = correct_text(text_with_errors)
print(corrected_text)

gonna play during sports


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## I made the basic and intial solution for spelling correction and included:
### N-gram Dataset
- **Combined Multiple Corpora**: We integrated multiple corpora, including Norvig's `big.txt`, to improve coverage and reduce data sparsity for n-grams. This ensures that the system has a robust vocabulary and context for better spell correction.

### Edit Operations
- **Included `edits2`**: By including second-level edits (`edits2`), the system can capture more complex corrections, such as fixing "nevew" to "never".
- **Keyboard Proximity in `edits1`**: Prioritizing edits based on keyboard proximity helps correct likely typos (e.g., "gice" → "give" instead of "rice").

### Tokenization
- **RegexpTokenizer**: We used `RegexpTokenizer` to handle contractions (e.g., "gonna" remains one word), ensuring that tokenization aligns with natural language usage.

### Smoothing
- **Linear Interpolation**: We applied linear interpolation of n-gram probabilities (trigram: 60%, bigram: 30%, unigram: 10%) to handle unseen contexts and improve the robustness of the model.

### Efficiency
- **Counter for Frequency Storage**: We used Python's `Counter` for efficient frequency storage. For larger datasets, a probabilistic structure like Bloom filters could be explored to further optimize memory usage and performance.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [48]:
pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.


In [51]:
import random
from spellchecker import SpellChecker
import nltk
from nltk.corpus import nps_chat

# Checked on data fronm this corpus
nltk.download('nps_chat')

# Noise generation
def add_noise(sentence, noise_level=0.2):
    words = sentence.split()
    noisy = []
    for word in words:
        if len(word) > 3 and random.random() < noise_level:
            idx = random.randint(0, len(word)-1)
            noisy_word = word[:idx] + word[idx+1:]
            noisy.append(noisy_word)
        else:
            noisy.append(word)
    return ' '.join(noisy)

# Norvig
def norvig_correct_word(word):
    spell = SpellChecker()
    corrected_word = spell.correction(word)
    return corrected_word if corrected_word is not None else word

all_posts = list(nps_chat.posts())
test_posts = random.sample(all_posts, 100)

norvig_correct = 0
context_correct = 0
total = 0

for post in test_posts:
    sentence = " ".join(post)
    noisy = add_noise(sentence)
    context_fixed = correct_text(noisy)
    norvig_fixed = ' '.join([norvig_correct_word(word) for word in noisy.split()]) 
    original_words = sentence.split()
    context_words = context_fixed.split()
    norvig_words = norvig_fixed.split()
    for cw, nw, ow in zip(context_words, norvig_words, original_words):
        total += 1
        if cw == ow:
            context_correct += 1
        if nw == ow:
            norvig_correct += 1

print(f"Context-aware accuracy: {context_correct/total:.2f}")
print(f"Norvig's accuracy: {norvig_correct/total:.2f}")

[nltk_data] Downloading package nps_chat to /usr/share/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
Context-aware accuracy: 0.44
Norvig's accuracy: 0.75


#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)

## As my solution showed not such good results there are proposed improvements for a solution

### 1. **Enhanced Edit Operations**
   - **Levenshtein Distance-Based Edits**: In addition to keyboard proximity,implementaion Levenshtein distance (edit distance) can be useful for identifying common typographical errors that are not strictly adjacent on the keyboard but might be phonetically similar.

### 2. **Improved Tokenization**
   - **Subword Tokenization**: Utilization of subword tokenization (e.g., Byte-Pair Encoding, WordPiece) to break down words into smaller, meaningful units. This could improve the handling of rare or unseen words, as the model could learn to correct them based on their subword components.
   - **Contextual Tokenization**: Implementation tokenizers that dynamically adjust based on sentence context.

### 3. **Smoothing Techniques**
   - **Kneser-Ney Smoothing**: Aplication of Kneser-Ney smoothing to improve the estimation of low-frequency n-grams, making the system more robust against unseen or rare word combinations.
   - **Perplexity-Based Smoothing**: Use of perplexity to measure the model’s predictive uncertainty and adjust the smoothing accordingly. This can help fine-tune how much weight should be given to various n-gram models during correction.
ress the cases where the model consistently mispredicts certain words or phrases.